In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from classificatiom_model import EarlyStopping, ModelSaver
from classificatiom_model import plot_learning_loss_curves,train_regression_model,evaluate_regression_model
import os

In [2]:
# 加载加利福尼亚房价数据集
housing = fetch_california_housing()  # 从sklearn加载加利福尼亚房价数据集
X = housing.data  # 获取特征数据
y = housing.target  # 获取目标变量（房价）

# 数据拆分：训练集(60%)、验证集(20%)、测试集(20%)
# 首先将数据分为训练集(80%)和测试集(20%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 使用80%/20%的比例划分训练验证集和测试集

# 然后将训练集再分为训练集(75%，即总数据的60%)和验证集(25%，即总数据的20%)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 将训练验证集进一步划分为训练集和验证集

# 打印数据集大小
print(f"训练集大小: {X_train.shape[0]} 样本")  # 显示训练集样本数量
print(f"验证集大小: {X_val.shape[0]} 样本")  # 显示验证集样本数量
print(f"测试集大小: {X_test.shape[0]} 样本")  # 显示测试集样本数量

# 标准化特征
scaler = StandardScaler()  # 创建标准化转换器
X_train_scaled = scaler.fit_transform(X_train)  # 对训练集进行拟合和转换
X_val_scaled = scaler.transform(X_val)  # 使用训练集的参数对验证集进行转换
X_test_scaled = scaler.transform(X_test)  # 使用训练集的参数对测试集进行转换


# %% cell 3 code

# 自定义数据集类
class HousingDataset(Dataset):
    """
    加利福尼亚房价数据集的PyTorch数据集类
    
    参数:
        features: 特征数据
        targets: 目标值
        wide_indices: 用于wide部分的特征索引
        deep_indices: 用于deep部分的特征索引
    """
    def __init__(self, features, targets):
        self.features = torch.FloatTensor(features)  # 将特征转换为PyTorch张量
        self.targets = torch.FloatTensor(targets).view(-1, 1)  # 确保目标是二维的
    
    def __len__(self):
        return len(self.features)  # 返回数据集的大小
    
    def __getitem__(self, idx):
        # 返回wide特征、deep特征和目标值
        return self.features[idx], self.targets[idx]  # 返回指定索引的特征和目标

# 创建数据集实例
# 这里我们假设所有特征都用于wide和deep部分
train_dataset = HousingDataset(X_train_scaled, y_train)  # 创建训练数据集
val_dataset = HousingDataset(X_val_scaled, y_val)  # 创建验证数据集
test_dataset = HousingDataset(X_test_scaled, y_test)  # 创建测试数据集

# 创建数据加载器
batch_size = 64  # 设置批次大小
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # 创建训练数据加载器，打乱数据
val_loader = DataLoader(val_dataset, batch_size=batch_size)  # 创建验证数据加载器
test_loader = DataLoader(test_dataset, batch_size=batch_size)  # 创建测试数据加载器

# 检查数据加载器是否正常工作
sample_batch = next(iter(train_loader))  # 获取一个批次的样本
print(f"批次中的特征形状: {[x.shape for x in sample_batch[0]]}")  # 打印特征的形状
print(f"批次中的目标形状: {sample_batch[1].shape}")  # 打印目标的形状


训练集大小: 12384 样本
验证集大小: 4128 样本
测试集大小: 4128 样本
批次中的特征形状: [torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Size([8]), torch.Si

In [4]:
from torch import optim  # 导入PyTorch的优化器模块



# 定义神经网络模型
class RegressionModel(nn.Module):
    """
    简单的回归神经网络模型，用于预测房价
    
    参数:
        input_dim: 输入特征的维度
    """
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 30)  # 第一个全连接层，将输入特征映射到30个神经元
        self.activation = nn.ReLU()  # ReLU激活函数，引入非线性
        self.output = nn.Linear(30, 1)  # 输出层，将30个神经元映射到1个输出（房价预测值）
        
    def forward(self, x):
        x = self.activation(self.layer1(x))  # 通过第一层后应用ReLU激活函数
        x = self.output(x)  # 通过输出层得到最终预测结果
        return x  # 返回预测的房价值


# 初始化模型、损失函数和优化器
input_dim = X_train.shape[1]
model = RegressionModel(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# 打印模型结构
print(model)

RegressionModel(
  (layer1): Linear(in_features=8, out_features=30, bias=True)
  (activation): ReLU()
  (output): Linear(in_features=30, out_features=1, bias=True)
)


In [5]:
# 评估模型
def evaluate_regression_model(model, dataloader, device, criterion):
    """
    评估回归模型在给定数据集上的性能
    
    参数:
        model: 待评估的神经网络模型
        dataloader: 数据加载器，包含要评估的数据
        device: 计算设备（CPU或GPU）
        criterion: 损失函数
        
    返回:
        平均损失值
    """
    model.eval()  # 将模型设置为评估模式，关闭dropout等训练特有操作
    running_loss = 0.0  # 初始化累计损失值
    
    with torch.no_grad():  # 禁止 autograd 记录计算图，节省显存与算力
        for inputs, targets in dataloader:  # 遍历数据加载器中的每个批次
            inputs, targets = inputs.to(device), targets.to(device)  # 将输入和目标数据移至指定设备
            outputs = model(inputs)  # 前向计算，获取模型预测结果
            loss = criterion(outputs, targets)  # 计算预测结果与真实值之间的损失
            
            running_loss += loss.item() * inputs.size(0)  # 累加批次损失（乘以批次大小以获得总损失）
    
    return running_loss / len(dataloader.dataset)  # 返回平均损失（总损失除以样本数量）

In [10]:
# 对学习率进行网格搜索
learning_rates = [0.001, 0.005, 0.01, 0.05, 0.1]  # 定义要搜索的学习率列表
results = {}  # 存储不同学习率的结果
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 将模型和损失函数移动到指定设备
criterion = criterion.to(device)

# 遍历不同的学习率进行训练
for lr in learning_rates:
    print(f"正在训练学习率为 {lr} 的模型...")
    
    # 重新初始化模型
    model = RegressionModel(input_dim).to(device)
    
    # 使用当前学习率创建优化器
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # 初始化早停和模型保存对象
    early_stopping = EarlyStopping(patience=10, verbose=True)
    model_saver = ModelSaver(save_dir=f'model_weights_lr_{lr}')
    
    # 训练模型
    trained_model, record_dict = train_regression_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=100,
        print_every=10,
        eval_step=500,
        early_stopping=early_stopping,
        model_saver=model_saver,
        device=device.type  # 传递设备类型字符串而不是设备对象
    )
    
    # 在验证集上评估模型
    val_loss = evaluate_regression_model(trained_model, val_loader, device, criterion)
    results[lr] = {
        'model': trained_model,
        'record_dict': record_dict,
        'val_loss': val_loss
    }
    print(f"学习率 {lr} 的验证集损失: {val_loss:.4f}")

# 找出最佳学习率
best_lr = min(results.keys(), key=lambda x: results[x]['val_loss'])
print(f"最佳学习率: {best_lr}, 验证集损失: {results[best_lr]['val_loss']:.4f}")

# 使用最佳模型和记录字典进行后续操作
model = results[best_lr]['model']
record_dict = results[best_lr]['record_dict']


正在训练学习率为 0.001 的模型...


train progress:   0%|          | 0/19400 [00:00<?, ?it/s]

学习率 0.001 的验证集损失: 0.3265
正在训练学习率为 0.005 的模型...


train progress:   0%|          | 0/19400 [00:00<?, ?it/s]

学习率 0.005 的验证集损失: 0.2958
正在训练学习率为 0.01 的模型...


...

早停触发! 最佳验证准确率: -0.2879
早停: 已有10轮验证损失没有改善！
学习率 0.01 的验证集损失: 0.3685
正在训练学习率为 0.05 的模型...


train progress:   0%|          | 0/19400 [00:00<?, ?it/s]

早停触发! 最佳验证准确率: -0.3382
早停: 已有10轮验证损失没有改善！
学习率 0.05 的验证集损失: 0.3534
正在训练学习率为 0.1 的模型...


train progress:   0%|          | 0/19400 [00:00<?, ?it/s]

早停触发! 最佳验证准确率: -0.3799
早停: 已有10轮验证损失没有改善！
学习率 0.1 的验证集损失: 0.4296
最佳学习率: 0.005, 验证集损失: 0.2958


In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
test_loss=evaluate_regression_model(model,test_loader,device,criterion)
print(f"测试集上的损失为{test_loss:.4f}")

测试集上的损失为0.2965
